In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
import math

Loading BokehJS ...

In [2]:
df = yf.Ticker('VOO').history(interval ='1wk' , period ='120mo')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 523 entries, 2014-03-03 00:00:00-05:00 to 2024-03-04 00:00:00-05:00
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           523 non-null    float64
 1   High           523 non-null    float64
 2   Low            523 non-null    float64
 3   Close          523 non-null    float64
 4   Volume         523 non-null    int64  
 5   Dividends      523 non-null    float64
 6   Stock Splits   523 non-null    float64
 7   Capital Gains  523 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 36.8 KB


In [3]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2014-03-03 00:00:00-05:00,144.256942,144.750237,143.629883,144.156616,2242600,0.000,0.0,0.0
2014-03-10 00:00:00-04:00,144.006152,144.574679,141.297253,141.489548,5878900,0.000,0.0,0.0
2014-03-17 00:00:00-04:00,142.183535,144.800465,142.125016,143.320602,5284800,0.000,0.0,0.0
2014-03-24 00:00:00-04:00,143.128308,143.504540,140.862529,142.049759,5892200,0.779,0.0,0.0
2014-03-31 00:00:00-04:00,143.655706,145.990627,143.227364,143.403748,7031400,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-02-05 00:00:00-05:00,453.730011,461.029999,450.540009,460.670013,26436100,0.000,0.0,0.0
2024-02-12 00:00:00-05:00,460.609985,462.739990,451.000000,459.029999,29562300,0.000,0.0,0.0
2024-02-19 00:00:00-05:00,457.440002,468.869995,453.600006,466.779999,17909000,0.000,0.0,0.0


In [4]:
class DCA(Strategy):

    amount_to_invest = 100

    def init(self):
        self.month = self.I(lambda x: x , self.data.Close.s.index.month)

    def next(self):
        if self.month[-1] != self.month[-2]: 
            self.buy( size = math.floor(self.amount_to_invest / self.data.Close[-1]))
                     
df= df*10**-6            
bt = Backtest(df,DCA,trade_on_close = True, cash = 100000 )
stats = bt.run()
bt.plot()
print(stats)

Start                     2014-03-03 00:00...
End                       2024-03-04 00:00...
Duration                   3654 days 00:00:00
Exposure Time [%]                   98.852772
Equity Final [$]                112191.135206
Equity Peak [$]                 112191.788704
Return [%]                          12.191135
Buy & Hold Return [%]              224.903021
Return (Ann.) [%]                    5.699225
Volatility (Ann.) [%]                4.274471
Sharpe Ratio                         1.333317
Sortino Ratio                        2.049181
Calmar Ratio                         1.336957
Max. Drawdown [%]                   -4.262833
Avg. Drawdown [%]                   -0.274699
Max. Drawdown Duration      693 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                  119
Win Rate [%]                            100.0
Best Trade [%]                     237.601363
Worst Trade [%]                      2.335724
Avg. Trade [%]                    

In [5]:
trades = stats["_trades"]
price_paid = trades["Size"] * trades["EntryPrice"]
total_invested = price_paid.sum()

current_shares = trades["Size"].sum()
current_equity = current_shares * df.Close.iloc[-1]

print("Total investment:",f'{total_invested} usd')
print("Current Shares:",current_shares / (10**6))
print("current Equity:", f'{current_equity} usd')

print("Return:", f'{current_equity*100 / total_invested :.3f} %')

Total investment: 11899.983427242767 usd
Current Shares: 51.103605
current Equity: 23935.354675805967 usd
Return: 201.138 %
